In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv('data/model_data.csv')

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41185 entries, 0 to 41184
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               41185 non-null  int64  
 1   job               41185 non-null  object 
 2   marital           41185 non-null  object 
 3   education         41185 non-null  object 
 4   default           41185 non-null  object 
 5   housing           41185 non-null  object 
 6   loan              41185 non-null  object 
 7   contact           41185 non-null  object 
 8   month             41185 non-null  object 
 9   day_of_week       41185 non-null  object 
 10  campaign          41185 non-null  int64  
 11  contacted_before  41185 non-null  int64  
 12  previous          41185 non-null  int64  
 13  poutcome          41185 non-null  object 
 14  emp.var.rate      41185 non-null  float64
 15  cons.price.idx    41185 non-null  int64  
 16  cons.conf.idx     41185 non-null  int64 

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,contacted_before,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,subscribed
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,0,0,nonexistent,3.0,3,2,3,2,0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,0,0,nonexistent,3.0,3,2,3,2,0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,0,0,nonexistent,3.0,3,2,3,2,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,0,0,nonexistent,3.0,3,2,3,2,0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,0,0,nonexistent,3.0,3,2,3,2,0


In [3]:
from pycaret.classification import *
s = (setup(data, target = 'subscribed',
            session_id = 420,
            fix_imbalance=True,
            categorical_features=['job','marital','education','default','housing','loan','contact','poutcome'],
            ignore_features=['month','day_of_week'],
            remove_multicollinearity = True,
            multicollinearity_threshold=0.8,
            feature_selection=True,
            feature_selection_method='univariate',
            n_features_to_select=15,
            fold=5
            ))

,Description,Value
0,Session id,420
1,Target,subscribed
2,Target type,Binary
3,Original data shape,"(41185, 20)"
4,Transformed data shape,"(63518, 16)"
5,Transformed train set shape,"(51162, 16)"
6,Transformed test set shape,"(12356, 16)"
7,Ignore features,2
8,Ordinal features,3
9,Numeric features,9


In [6]:
#s.dataset_transformed.info()
s.train_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51162 entries, 5921 to 63517
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   job_blue-collar              51162 non-null  float64
 1   job_retired                  51162 non-null  float64
 2   job_student                  51162 non-null  float64
 3   marital_single               51162 non-null  float64
 4   education_university.degree  51162 non-null  float64
 5   education_basic.9y           51162 non-null  float64
 6   default                      51162 non-null  float64
 7   contact                      51162 non-null  float64
 8   campaign                     51162 non-null  float64
 9   contacted_before             51162 non-null  float64
 10  previous                     51162 non-null  float64
 11  poutcome_nonexistent         51162 non-null  float64
 12  emp.var.rate                 51162 non-null  float64
 13  cons.price.id

In [5]:
# functional API
best = compare_models(include=['lr','rf','lightgbm'], n_select=3)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.8872,0.7811,0.3682,0.4996,0.4237,0.3628,0.3681,0.8460
rf,Random Forest Classifier,0.8804,0.7518,0.3402,0.4594,0.3903,0.3257,0.3305,48.5320
lr,Logistic Regression,0.7449,0.7713,0.6807,0.2594,0.3756,0.2539,0.3006,1.6940


In [ ]:

evaluate_model(best[1])